In [1]:
from simulator import k_sequence_WH_worker
from network_statistics import WH_cmp
from numpy import log, floor
from tqdm import tqdm
from multiprocessing import Pool
WH = [(m, K) for K in range(10, 311, 20) for m in range(1, 3*int(floor(log(K))), 2)]
WH = [x for i, x in enumerate(WH) if not any((WH_cmp(x, y) and WH_cmp(y, x) for y in WH[:i]))]

print(WH)

[(1, 10), (3, 10), (5, 10), (1, 30), (3, 30), (5, 30), (7, 30), (1, 50), (3, 50), (5, 50), (7, 50), (1, 70), (3, 70), (5, 70), (7, 70), (9, 70), (11, 70), (1, 90), (3, 90), (5, 90), (7, 90), (9, 90), (11, 90), (1, 110), (3, 110), (5, 110), (7, 110), (9, 110), (11, 110), (1, 130), (3, 130), (5, 130), (7, 130), (9, 130), (11, 130), (1, 150), (3, 150), (5, 150), (7, 150), (9, 150), (11, 150), (13, 150), (1, 170), (3, 170), (5, 170), (7, 170), (9, 170), (11, 170), (13, 170), (1, 190), (3, 190), (5, 190), (7, 190), (9, 190), (11, 190), (13, 190), (1, 210), (3, 210), (5, 210), (7, 210), (9, 210), (11, 210), (13, 210), (1, 230), (3, 230), (5, 230), (7, 230), (9, 230), (11, 230), (13, 230), (1, 250), (3, 250), (5, 250), (7, 250), (9, 250), (11, 250), (13, 250), (1, 270), (3, 270), (5, 270), (7, 270), (9, 270), (11, 270), (13, 270), (1, 290), (3, 290), (5, 290), (7, 290), (9, 290), (11, 290), (13, 290), (1, 310), (3, 310), (5, 310), (7, 310), (9, 310), (11, 310), (13, 310)]


In [2]:
pool = Pool()

for _ in tqdm(pool.imap_unordered(k_sequence_WH_worker, WH), total=len(WH)):
    pass
pool.close()
pool.join()

 98%|█████████▊| 96/98 [25:31<07:39, 229.78s/it]Process ForkPoolWorker-10:
Process ForkPoolWorker-7:
Process ForkPoolWorker-5:
Process ForkPoolWorker-9:
Process ForkPoolWorker-12:
Process ForkPoolWorker-6:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py",

KeyboardInterrupt: 